<a href="https://colab.research.google.com/github/OHDSI/ClinicalTrialsWGETL/blob/master/notebooks/phuse_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preparation steps**

In [1]:
#@title **This block of code will install PosgreSQL**
%%capture
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" >/etc/apt/sources.list.d/pgdg.list
!apt -qq update
!apt -yq install postgresql-12 postgresql-client-12
!service postgresql start
# make calling psql shorter
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"  
!psql postgres -c "CREATE DATABASE root"  # now just !psql -c "..."
# load SQL extensions
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [2]:
#@title **Download the PHUSE data to local environment**

!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/ae.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/cm.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/dm.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/ds.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/ex.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/lbch.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/lbhe.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/lbur.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/mh.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/qsco.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/qsda.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/qsgi.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/qshi.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/qsmm.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/qsni.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/relrec.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/sc.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/se.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/suppae.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/suppdm.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/suppds.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/supplbch.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/supplbhe.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/supplbur.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/sv.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/ta.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/te.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/ti.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/ts.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/tv.csv
!wget -q https://raw.githubusercontent.com/OHDSI/ClinicalTrialsWGETL/master/data/source/csv/vs.csv

!mkdir -p source
!mv *.csv source/

In [3]:
#@title **Check the files**
!ls source

ae.csv	lbch.csv  qsda.csv  relrec.csv	suppds.csv    ta.csv  vs.csv
cm.csv	lbhe.csv  qsgi.csv  sc.csv	supplbch.csv  te.csv
dm.csv	lbur.csv  qshi.csv  se.csv	supplbhe.csv  ti.csv
ds.csv	mh.csv	  qsmm.csv  suppae.csv	supplbur.csv  ts.csv
ex.csv	qsco.csv  qsni.csv  suppdm.csv	sv.csv	      tv.csv


In [4]:
#@title **Create source tables**
%%sql
DROP SCHEMA IF EXISTS src CASCADE;

CREATE SCHEMA src;

-- DROP TABLE IF EXISTS src.ae;
CREATE TABLE src.ae
(
    studyid     VARCHAR(12),
    domain      VARCHAR(2),
    usubjid     VARCHAR(11),
    aeseq       NUMERIC,
    aespid      VARCHAR(3),
    aeterm      VARCHAR(200),
    aellt       VARCHAR(100),
    aelltcd     INTEGER,
    aedecod     VARCHAR(200),
    aeptcd      INTEGER,
    aehlt       VARCHAR(100),
    aehltcd     INTEGER,
    aehlgt      VARCHAR(100),
    aehlgtcd    INTEGER,
    aebodsys    VARCHAR(67),
    aebdsycd    INTEGER,
    aesoc       VARCHAR(100),
    aesoccd     INTEGER,
    aesev       VARCHAR(8),
    aeser       VARCHAR(1),
    aeacn       VARCHAR(30),
    aerel       VARCHAR(8),
    aeout       VARCHAR(200),
    aescan      VARCHAR(1),
    aescong     VARCHAR(1),
    aesdisab    VARCHAR(1),
    aesdth      VARCHAR(1),
    aeshosp     VARCHAR(1),
    aeslife     VARCHAR(1),
    aesod       VARCHAR(1),
    epoch       VARCHAR(9),
    aedtc       VARCHAR(10),
    aestdtc     VARCHAR(10),
    aeendtc     VARCHAR(10),
    aedy        INTEGER,
    aestdy      INTEGER,
    aeendy      INTEGER
);

COPY
    src.ae(
        studyid,
        domain,
        usubjid,
        aeseq,
        aespid,
        aeterm,
        aellt,
        aelltcd,
        aedecod,
        aeptcd,
        aehlt,
        aehltcd,
        aehlgt,
        aehlgtcd,
        aebodsys,
        aebdsycd,
        aesoc,
        aesoccd,
        aesev,
        aeser,
        aeacn,
        aerel,
        aeout,
        aescan,
        aescong,
        aesdisab,
        aesdth,
        aeshosp,
        aeslife,
        aesod,
        epoch,
        aedtc,
        aestdtc,
        aeendtc,
        aedy,
        aestdy,
        aeendy
        )
FROM '/content/source/ae.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.cm;
CREATE TABLE src.cm
(
    studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	cmseq		NUMERIC,
	cmspid		VARCHAR(2),
	cmtrt		VARCHAR(44),
	cmdecod		VARCHAR(24),
	cmindc		VARCHAR(34),
	cmclas		VARCHAR(42),
	cmdose		NUMERIC,
	cmdosu		VARCHAR(17),
	cmdosfrq	VARCHAR(15),
	cmroute		VARCHAR(200),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	cmdtc		VARCHAR(10),  -- for incomplete dates
	cmstdtc		VARCHAR(10),  -- for incomplete dates
	cmendtc		VARCHAR(10),  -- for incomplete dates
	cmdy		INTEGER,
	cmstdy		INTEGER,
	cmendy	    INTEGER
);

COPY
    src.cm(
        studyid,
    	domain,
    	usubjid,
    	cmseq,
    	cmspid,
    	cmtrt,
    	cmdecod,
    	cmindc,
    	cmclas,
    	cmdose,
    	cmdosu,
    	cmdosfrq,
    	cmroute,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	cmdtc,
    	cmstdtc,
    	cmendtc,
    	cmdy,
    	cmstdy,
    	cmendy
    )
FROM '/content/source/cm.csv' DELIMITER ',' CSV HEADER; 


-- DROP TABLE IF EXISTS src.dm;
CREATE TABLE src.dm
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	subjid		VARCHAR(4),
	rfstdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	rfendtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	rfxstdtc	VARCHAR(20),  -- instead of TIMESTAMP for possible incomplete dates
	rfxendtc	VARCHAR(20),  -- instead of TIMESTAMP for possible incomplete dates
	rficdtc		VARCHAR(20),  -- instead of TIMESTAMP for possible incomplete dates
	rfpendtc	VARCHAR(20),  -- instead of TIMESTAMP for possible incomplete dates
	dthdtc		VARCHAR(20),  -- instead of TIMESTAMP for possible incomplete dates
	dthfl		VARCHAR(1),
	siteid		VARCHAR(3),
	age			INTEGER,
	ageu		VARCHAR(6),
	sex			VARCHAR(1),
	race		VARCHAR(78),
	ethnic		VARCHAR(25),
	armcd		VARCHAR(8),
	arm			VARCHAR(20),
	actarmcd	VARCHAR(8),
	actarm		VARCHAR(20),
	country		VARCHAR(3),
	dmdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	dmdy		INTEGER
);

COPY
    src.dm(
        studyid,
    	domain,
    	usubjid,
    	subjid,
    	rfstdtc,
    	rfendtc,
    	rfxstdtc,
    	rfxendtc,
    	rficdtc,
    	rfpendtc,
    	dthdtc,
    	dthfl,
    	siteid,
    	age,
    	ageu,
    	sex,
    	race,
    	ethnic,
    	armcd,
    	arm,
    	actarmcd,
    	actarm,
    	country,
    	dmdtc,
    	dmdy
    )
FROM '/content/source/dm.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.ds;
CREATE TABLE src.ds
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	dsseq		NUMERIC,
	dsspid		VARCHAR(2),
	dsterm		VARCHAR(63),
	dsdecod		VARCHAR(63),
	dscat		VARCHAR(17),
	visitnum	NUMERIC,
	visit		VARCHAR(17),
	epoch		VARCHAR(9),
	dsdtc		VARCHAR(20),  -- instead of TIMESTAMP for possible incomplete dates
	dsstdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	dsdy		INTEGER,
	dsstdy		INTEGER
);

COPY
    src.ds(
        studyid,
    	domain,
    	usubjid,
    	dsseq,
    	dsspid,
    	dsterm,
    	dsdecod,
    	dscat,
    	visitnum,
    	visit,
    	epoch,
    	dsdtc,
    	dsstdtc,
    	dsdy,
    	dsstdy
    )
FROM '/content/source/ds.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.ex;
CREATE TABLE src.ex (
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	exseq		NUMERIC,  -- in define it's INTEGER
	extrt		VARCHAR(10),
	exdose		NUMERIC,  -- in define it's INTEGER
	exdosu		VARCHAR(2),
	exdosfrm	VARCHAR(5),
	exdosfrq	VARCHAR(2),
	exroute		VARCHAR(11),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	exstdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	exendtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	exstdy		INTEGER,
	exendy		INTEGER
);

COPY
    src.ex(
        studyid,
    	domain,
    	usubjid,
    	exseq,
    	extrt,
    	exdose,
    	exdosu,
    	exdosfrm,
    	exdosfrq,
    	exroute,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	exstdtc,
    	exendtc,
    	exstdy,
    	exendy
    )
FROM '/content/source/ex.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.lbch;
CREATE TABLE src.lbch
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	lbseq		INTEGER,  -- in define it's INTEGER
	lbtestcd	VARCHAR(7),
	lbtest		VARCHAR(200),
	lbcat		VARCHAR(10),
	lborres		VARCHAR(5),
	lborresu	VARCHAR(8),
	lbornrlo	VARCHAR(5),  -- in defite it's text with lenght 5
	lbornrhi	VARCHAR(7),
	lbstresc	VARCHAR(8),
	lbstresn	NUMERIC,  -- in define it's INTEGER
	lbstresu	VARCHAR(8),
	lbstnrlo	NUMERIC,  -- in define it's INTEGER
	lbstnrhi	NUMERIC,  -- in define it's INTEGER
	lbnrind		VARCHAR(200),
	lbblfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	lbdtc		VARCHAR(16),  -- instead of TIMESTAMP for possible incomplete dates
	lbdy		INTEGER
);

COPY
    src.lbch(
        studyid,
    	domain,
    	usubjid,
    	lbseq,
    	lbtestcd,
    	lbtest,
    	lbcat,
    	lborres,
    	lborresu,
    	lbornrlo,
    	lbornrhi,
    	lbstresc,
    	lbstresn,
    	lbstresu,
    	lbstnrlo,
    	lbstnrhi,
    	lbnrind,
    	lbblfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	lbdtc,
    	lbdy
    )
FROM '/content/source/lbch.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.lbhe;
CREATE TABLE src.lbhe
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	lbseq		INTEGER,  -- in define it's INTEGER
	lbtestcd	VARCHAR(7),
	lbtest		VARCHAR(200),
	lbcat		VARCHAR(10),
	lborres		VARCHAR(5),
	lborresu	VARCHAR(8),
	lbornrlo	VARCHAR(5),
	lbornrhi	VARCHAR(7),
	lbstresc	VARCHAR(8),
	lbstresn	NUMERIC,  -- in define it's INTEGER
	lbstresu	VARCHAR(8),
	lbstnrlo	NUMERIC,  -- in define it's INTEGER
	lbstnrhi	NUMERIC,  -- in define it's INTEGER
	lbnrind		VARCHAR(200),
	lbblfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	lbdtc		VARCHAR(16),  -- instead of TIMESTAMP for possible incomplete dates
	lbdy		INTEGER
);

COPY
    src.lbhe
    (
        studyid,
    	domain,
    	usubjid,
    	lbseq,
    	lbtestcd,
    	lbtest,
    	lbcat,
    	lborres,
    	lborresu,
    	lbornrlo,
    	lbornrhi,
    	lbstresc,
    	lbstresn,
    	lbstresu,
    	lbstnrlo,
    	lbstnrhi,
    	lbnrind,
    	lbblfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	lbdtc,
    	lbdy
    )
FROM '/content/source/lbhe.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.lbur;
CREATE TABLE src.lbur
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	lbseq		NUMERIC,  -- in define it's INTEGER
	lbtestcd	VARCHAR(7),
	lbtest		VARCHAR(200),
	lbcat		VARCHAR(10),
	lborres		VARCHAR(5),
	lborresu	VARCHAR(8),
	lbornrlo	VARCHAR(5),
	lbornrhi	VARCHAR(7),
	lbstresc	VARCHAR(8),
	lbstresn	NUMERIC,  -- in define it's INTEGER
	lbstresu	VARCHAR(8),
	lbstnrlo	NUMERIC,  -- in define it's INTEGER
	lbstnrhi	NUMERIC,  -- in define it's INTEGER
	lbnrind		VARCHAR(200),
	lbblfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	lbdtc		VARCHAR(16),  -- instead of TIMESTAMP for possible incomplete dates
	lbdy		INTEGER
);

COPY
    src.lbur
    (
        studyid,
    	domain,
    	usubjid,
    	lbseq,
    	lbtestcd,
    	lbtest,
    	lbcat,
    	lborres,
    	lborresu,
    	lbornrlo,
    	lbornrhi,
    	lbstresc,
    	lbstresn,
    	lbstresu,
    	lbstnrlo,
    	lbstnrhi,
    	lbnrind,
    	lbblfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	lbdtc,
    	lbdy
    )
FROM '/content/source/lbur.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.mh;
CREATE TABLE src.mh
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	mhseq		NUMERIC,  -- in define it's INTEGER
	mhspid		VARCHAR(3),
	mhterm		VARCHAR(19),
	mhllt		VARCHAR(200),
	mhdecod		VARCHAR(44),
	mhhlt		VARCHAR(200),
	mhhlgt		VARCHAR(200),
	mhcat		VARCHAR(34),
	mhbodsys	VARCHAR(67),
	mhsev		VARCHAR(8),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	mhdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	mhstdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	mhdy		INTEGER
);

COPY
    src.mh
    (
        studyid,
    	domain,
    	usubjid,
    	mhseq,
    	mhspid,
    	mhterm,
    	mhllt,
    	mhdecod,
    	mhhlt,
    	mhhlgt,
    	mhcat,
    	mhbodsys,
    	mhsev,
    	visitnum,
    	visit,
    	visitdy,
    	mhdtc,
    	mhstdtc,
    	mhdy
    )
FROM '/content/source/mh.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.qsco;
CREATE TABLE src.qsco
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	qsseq		NUMERIC,  -- in define it's INTEGER
	qstestcd	VARCHAR(8),
	qstest		VARCHAR(40),
	qscat		VARCHAR(70),
	qsscat		VARCHAR(26),
	qsorres		VARCHAR(20),
	qsorresu	VARCHAR(7),
	qsstresc	VARCHAR(4),
	qsstresn	NUMERIC,  -- in define it's INTEGER
	qsstresu	VARCHAR(7),
	qsstat		VARCHAR(8),
	qsreasnd	VARCHAR(17),
	qsblfl		VARCHAR(1),
	qsdrvfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	qsdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	qsdy		INTEGER
);

COPY
    src.qsco
    (
        studyid,
    	domain,
    	usubjid,
    	qsseq,
    	qstestcd,
    	qstest,
    	qscat,
    	qsscat,
    	qsorres,
    	qsorresu,
    	qsstresc,
    	qsstresn,
    	qsstresu,
    	qsstat,
    	qsreasnd,
    	qsblfl,
    	qsdrvfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	qsdtc,
    	qsdy
    )
FROM '/content/source/qsco.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.qsda;
CREATE TABLE src.qsda
(
    studyid		VARCHAR(12),
    domain		VARCHAR(2),
    usubjid		VARCHAR(11),
    qsseq		NUMERIC,-- in define it's INTEGER
    qstestcd	VARCHAR(8),
    qstest		VARCHAR(40),
    qscat		VARCHAR(70),
    qsscat		VARCHAR(26),
    qsorres		VARCHAR(20),
    qsorresu	VARCHAR(7),
    qsstresc	VARCHAR(4),
    qsstresn	NUMERIC,  -- in define it's INTEGER
    qsstresu	VARCHAR(7),
    qsstat		VARCHAR(8),
    qsreasnd	VARCHAR(17),
    qsblfl		VARCHAR(1),
    qsdrvfl		VARCHAR(1),
    visitnum	NUMERIC,
    visit		VARCHAR(19),
    visitdy		INTEGER,
    epoch		VARCHAR(9),
    qsdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
    qsdy		INTEGER
);

COPY
    src.qsda
    (
        studyid,
        domain,
        usubjid,
        qsseq,
        qstestcd,
        qstest,
        qscat,
        qsscat,
        qsorres,
        qsorresu,
        qsstresc,
        qsstresn,
        qsstresu,
        qsstat,
        qsreasnd,
        qsblfl,
        qsdrvfl,
        visitnum,
        visit,
        visitdy,
        epoch,
        qsdtc,
        qsdy
    )
FROM '/content/source/qsda.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.qsgi;
CREATE TABLE src.qsgi
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	qsseq		NUMERIC,  -- in define it's INTEGER
	qstestcd	VARCHAR(8),
	qstest		VARCHAR(40),
	qscat		VARCHAR(70),
	qsscat		VARCHAR(70),
	qsorres		VARCHAR(20),
	qsorresu	VARCHAR(7),
	qsstresc	VARCHAR(4),
	qsstresn	NUMERIC,  -- in define it's INTEGER
	qsstresu	VARCHAR(7),
	qsstat		VARCHAR(8),
	qsreasnd	VARCHAR(17),
	qsblfl		VARCHAR(1),
	qsdrvfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	qsdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	qsdy		INTEGER
);

COPY
    src.qsgi
    (
        studyid,
    	domain,
    	usubjid,
    	qsseq,
    	qstestcd,
    	qstest,
    	qscat,
    	qsscat,
    	qsorres,
    	qsorresu,
    	qsstresc,
    	qsstresn,
    	qsstresu,
    	qsstat,
    	qsreasnd,
    	qsblfl,
    	qsdrvfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	qsdtc,
    	qsdy
    )
FROM '/content/source/qsgi.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.qshi;
CREATE TABLE src.qshi
(
    studyid     VARCHAR(12),
    domain		VARCHAR(2),
    usubjid		VARCHAR(11),
    qsseq		NUMERIC,-- in define it's INTEGER
    qstestcd	VARCHAR(8),
    qstest		VARCHAR(40),
    qscat   	VARCHAR(70),
    qsscat		VARCHAR(26),
    qsorres		VARCHAR(20),
    qsorresu	VARCHAR(7),
    qsstresc	VARCHAR(4),
    qsstresn	NUMERIC,-- in define it's INTEGER
    qsstresu	VARCHAR(7),
    qsstat		VARCHAR(8),
    qsreasnd	VARCHAR(17),
    qsblfl		VARCHAR(1),
    qsdrvfl		VARCHAR(1),
    visitnum	NUMERIC,
    visit		VARCHAR(19),
    visitdy		INTEGER,
    epoch		VARCHAR(9),
    qsdtc       VARCHAR(10),-- instead of DATE for possible incomplete dates
    qsdy        INTEGER
);

COPY
    src.qshi
    (
        studyid,
        domain,
        usubjid,
        qsseq,
        qstestcd,
        qstest,
        qscat,
        qsscat,
        qsorres,
        qsorresu,
        qsstresc,
        qsstresn,
        qsstresu,
        qsstat,
        qsreasnd,
        qsblfl,
        qsdrvfl,
        visitnum,
        visit,
        visitdy,
        epoch,
        qsdtc,
        qsdy
    )
FROM '/content/source/qshi.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.qsmm;
CREATE TABLE src.qsmm
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	qsseq		NUMERIC,-- in define it's INTEGER
	qstestcd	VARCHAR(8),
	qstest		VARCHAR(40),
	qscat		VARCHAR(70),
	qsscat		VARCHAR(26),
	qsorres		VARCHAR(20),
	qsorresu	VARCHAR(7),
	qsstresc	VARCHAR(4),
	qsstresn	NUMERIC,-- in define it's INTEGER
	qsstresu	VARCHAR(7),
	qsstat		VARCHAR(8),
	qsreasnd	VARCHAR(17),
	qsblfl		VARCHAR(1),
	qsdrvfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	qsdtc		VARCHAR(10),-- instead of DATE for possible incomplete dates
	qsdy		INTEGER
);

COPY
    src.qsmm
    (
        studyid,
    	domain,
    	usubjid,
    	qsseq,
    	qstestcd,
    	qstest,
    	qscat,
    	qsscat,
    	qsorres,
    	qsorresu,
    	qsstresc,
    	qsstresn,
    	qsstresu,
    	qsstat,
    	qsreasnd,
    	qsblfl,
    	qsdrvfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	qsdtc,
    	qsdy
    )
FROM '/content/source/qsmm.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.qsni;
CREATE TABLE src.qsni
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	qsseq		NUMERIC,-- in define it's INTEGER
	qstestcd	VARCHAR(8),
	qstest		VARCHAR(40),
	qscat		VARCHAR(70),
	qsscat		VARCHAR(26),
	qsorres		VARCHAR(20),
	qsorresu	VARCHAR(7),
	qsstresc	VARCHAR(4),
	qsstresn	NUMERIC,-- in define it's INTEGER
	qsstresu	VARCHAR(7),
	qsstat		VARCHAR(8),
	qsreasnd	VARCHAR(17),
	qsblfl		VARCHAR(1),
	qsdrvfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	qsdtc		VARCHAR(10),-- instead of DATE for possible incomplete dates
	qsdy		INTEGER
);

COPY
    src.qsni
    (
        studyid,
    	domain,
    	usubjid,
    	qsseq,
    	qstestcd,
    	qstest,
    	qscat,
    	qsscat,
    	qsorres,
    	qsorresu,
    	qsstresc,
    	qsstresn,
    	qsstresu,
    	qsstat,
    	qsreasnd,
    	qsblfl,
    	qsdrvfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	qsdtc,
    	qsdy
    )
FROM '/content/source/qsni.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.relrec;
CREATE TABLE src.relrec
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	reltype		VARCHAR(30),
	relid		VARCHAR(200)
);

COPY
    src.relrec
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	reltype,
    	relid
    )
FROM '/content/source/relrec.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.sc;
CREATE TABLE src.sc
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	scseq		NUMERIC,-- in define it's INTEGER
	sctestcd	VARCHAR(8),
	sctest		VARCHAR(27),
	sccat		VARCHAR(9),
	scorres		VARCHAR(2),
	scorresu	VARCHAR(5),
	scstresc	VARCHAR(2),
	scstresn	NUMERIC,-- in define it's INTEGER
	scstresu	VARCHAR(5),
	scdtc		VARCHAR(10),-- instead of DATE for possible incomplete dates
	scdy		INTEGER
);

COPY
    src.sc
    (
        studyid,
    	domain,
    	usubjid,
    	scseq,
    	sctestcd,
    	sctest,
    	sccat,
    	scorres,
    	scorresu,
    	scstresc,
    	scstresn,
    	scstresu,
    	scdtc,
    	scdy
    )
FROM '/content/source/sc.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.se;
CREATE TABLE src.se
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	seseq		NUMERIC,  -- in define it's INTEGER
	etcd		VARCHAR(200),
	element		VARCHAR(200),
	seupdes		VARCHAR(200),
	epoch		VARCHAR(9),
	sestdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	seendtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	sestdy		INTEGER,
	seendy		INTEGER
);

COPY
    src.se
    (
        studyid,
    	domain,
    	usubjid,
    	seseq,
    	etcd,
    	element,
    	seupdes,
    	epoch,
    	sestdtc,
    	seendtc,
    	sestdy,
    	seendy
    )
FROM '/content/source/se.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.suppae;
CREATE TABLE src.suppae
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	qnam		VARCHAR(8),
	qlabel		VARCHAR(40),
	qval		VARCHAR(200),
	qorig		VARCHAR(200),
	qeval		VARCHAR(200)
);

COPY
    src.suppae
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	qnam,
    	qlabel,
    	qval,
    	qorig,
    	qeval
    )
FROM '/content/source/suppae.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.suppdm;
CREATE TABLE src.suppdm
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	qnam		VARCHAR(8),
	qlabel		VARCHAR(40),
	qval		VARCHAR(200),
	qorig		VARCHAR(200),
	qeval		VARCHAR(200)
);

COPY
    src.suppdm
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	qnam,
    	qlabel,
    	qval,
    	qorig,
    	qeval
    )
FROM '/content/source/suppdm.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.suppds;
CREATE TABLE src.suppds
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	qnam		VARCHAR(8),
	qlabel		VARCHAR(40),
	qval		VARCHAR(200),
	qorig		VARCHAR(200),
	qeval		VARCHAR(200)
);

COPY
    src.suppds
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	qnam,
    	qlabel,
    	qval,
    	qorig,
    	qeval
    )
FROM '/content/source/suppds.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.supplbch;
CREATE TABLE src.supplbch
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	qnam		VARCHAR(8),
	qlabel		VARCHAR(40),
	qval		VARCHAR(200),
	qorig		VARCHAR(200),
	qeval		VARCHAR(200)
);

COPY
    src.supplbch
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	qnam,
    	qlabel,
    	qval,
    	qorig,
    	qeval
    )
FROM '/content/source/supplbch.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.supplbhe;
CREATE TABLE src.supplbhe
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	qnam		VARCHAR(8),
	qlabel		VARCHAR(40),
	qval		VARCHAR(200),
	qorig		VARCHAR(200),
	qeval		VARCHAR(200)
);

COPY
    src.supplbhe
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	qnam,
    	qlabel,
    	qval,
    	qorig,
    	qeval
    )
FROM '/content/source/supplbhe.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.supplbur;
CREATE TABLE src.supplbur
(
	studyid		VARCHAR(12),
	rdomain		VARCHAR(2),
	usubjid		VARCHAR(11),
	idvar		VARCHAR(8),
	idvarval	VARCHAR(200),
	qnam		VARCHAR(8),
	qlabel		VARCHAR(40),
	qval		VARCHAR(200),
	qorig		VARCHAR(200),
	qeval		VARCHAR(200)
);

COPY
    src.supplbur
    (
        studyid,
    	rdomain,
    	usubjid,
    	idvar,
    	idvarval,
    	qnam,
    	qlabel,
    	qval,
    	qorig,
    	qeval
    )
FROM '/content/source/supplbur.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.sv;
CREATE TABLE src.sv
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		NUMERIC,  -- in define it's INTEGER
	epoch		VARCHAR(9),
	svstdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	svendtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	svstdy		INTEGER,
	svendy		INTEGER,
	svupdes		VARCHAR(51)
);

COPY
    src.sv
    (
        studyid,
    	domain,
    	usubjid,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	svstdtc,
    	svendtc,
    	svstdy,
    	svendy,
    	svupdes
    )
FROM '/content/source/sv.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.ta;
CREATE TABLE src.ta
(
    studyid     VARCHAR(12),
    domain      VARCHAR(2),
    armcd       VARCHAR(8),
    arm         VARCHAR(20),
    taetord     INTEGER,
    etcd        VARCHAR(200),
    element     VARCHAR(200),
    tabranch    VARCHAR(200),
    tatrans     VARCHAR(200),
    epoch       VARCHAR(200)
);

COPY
    src.ta
    (
        studyid,
        domain,
        armcd,
        arm,
        taetord,
        etcd,
        element,
        tabranch,
        tatrans,
        epoch
    )
FROM '/content/source/ta.csv' DELIMITER ',' CSV HEADER;

-- DROP TABLE IF EXISTS src.te;
CREATE TABLE src.te
(
    studyid     VARCHAR(12),
    domain      VARCHAR(2),
    etcd        VARCHAR(200),
    element     VARCHAR(200),
    testrl      VARCHAR(200),
    teenrl      VARCHAR(200),
    tedur       VARCHAR(200)
);

COPY
    src.te
    (
        studyid,
        domain,
        etcd,
        element,
        testrl,
        teenrl,
        tedur
    )
FROM '/content/source/te.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.ti;
CREATE TABLE src.ti
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	ietestcd	VARCHAR(16),
	ietest		VARCHAR(166),
	iecat		VARCHAR(9)
);

COPY
    src.ti
    (
        studyid,
    	domain,
    	ietestcd,
    	ietest,
    	iecat
    )
FROM '/content/source/ti.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.ts;
CREATE TABLE src.ts
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	tsseq		NUMERIC,  -- in define it's INTEGER
	tsparmcd	VARCHAR(200),
	tsparm		VARCHAR(200),
	tsval		VARCHAR(200),
	tsvalnf		VARCHAR(4),
	tsvalcd		VARCHAR(11),
	tsvcdref	VARCHAR(8),
	tsvcdver	VARCHAR(18)
);

COPY
    src.ts
    (
        studyid,
        domain,
        tsseq,
        tsparmcd,
        tsparm,
        tsval,
        tsvalnf,
        tsvalcd,
        tsvcdref,
        tsvcdver
    )
FROM '/content/source/ts.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.tv;
CREATE TABLE src.tv
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	visitnum	NUMERIC,
	visit		VARCHAR(90),
	visitdy		INTEGER,
	armcd		VARCHAR(8),
	tvstrl		VARCHAR(200),
	tvenrl		VARCHAR(200)
);

COPY
    src.tv
    (
        studyid,
    	domain,
    	visitnum,
    	visit,
    	visitdy,
    	armcd,
    	tvstrl,
    	tvenrl
    )
FROM '/content/source/tv.csv' DELIMITER ',' CSV HEADER;


-- DROP TABLE IF EXISTS src.vs;
CREATE TABLE src.vs
(
	studyid		VARCHAR(12),
	domain		VARCHAR(2),
	usubjid		VARCHAR(11),
	vsseq		NUMERIC,  -- in define it's INTEGER
	vstestcd	VARCHAR(6),
	vstest		VARCHAR(24),
	vspos		VARCHAR(8),
	vsorres		VARCHAR(5),
	vsorresu	VARCHAR(9),
	vsstresc	VARCHAR(6),
	vsstresn	NUMERIC, -- in define it's INTEGER
	vsstresu	VARCHAR(9),
	vsstat		VARCHAR(8),
	vsloc		VARCHAR(11),
	vsblfl		VARCHAR(1),
	visitnum	NUMERIC,
	visit		VARCHAR(19),
	visitdy		INTEGER,
	epoch		VARCHAR(9),
	vsdtc		VARCHAR(10),  -- instead of DATE for possible incomplete dates
	vsdy		INTEGER,
	vstpt		VARCHAR(30),
	vstptnum	INTEGER,
	vseltm		VARCHAR(4),
	vstptref	VARCHAR(16)
);

COPY
    src.vs
    (
        studyid,
    	domain,
    	usubjid,
    	vsseq,
    	vstestcd,
    	vstest,
    	vspos,
    	vsorres,
    	vsorresu,
    	vsstresc,
    	vsstresn,
    	vsstresu,
    	vsstat,
    	vsloc,
    	vsblfl,
    	visitnum,
    	visit,
    	visitdy,
    	epoch,
    	vsdtc,
    	vsdy,
    	vstpt,
    	vstptnum,
    	vseltm,
    	vstptref
    )
FROM '/content/source/vs.csv' DELIMITER ',' CSV HEADER;


 * postgresql+psycopg2://@/postgres


""


In [5]:
from google.colab import files

## Start date of AE (ae.aestdtc)

In [8]:
%%sql

SELECT CASE
         WHEN aestdtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
         WHEN aestdtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
         WHEN aestdtc ~* '[0-9]{4}' THEN 'YYYY'
         ELSE 'missing'
       end      AS date_status,
       Count(*) AS counts
FROM   src.ae
GROUP  BY CASE
            WHEN aestdtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
            WHEN aestdtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
            WHEN aestdtc ~* '[0-9]{4}' THEN 'YYYY'
            ELSE 'missing'
          end
ORDER  BY date_status; 

 * postgresql+psycopg2://@/postgres


,date_status,counts
0,YYYY,11
1,YYYY-MM,13
2,YYYY-MM-DD,976


## End date of AE (ae.aeendtc)

In [9]:
%%sql

SELECT CASE
         WHEN aeendtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
         WHEN aeendtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
         WHEN aeendtc ~* '[0-9]{4}' THEN 'YYYY'
         ELSE 'missing'
       end      AS date_status,
       Count(*) AS counts
FROM   src.ae
GROUP  BY CASE
            WHEN aeendtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
            WHEN aeendtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
            WHEN aeendtc ~* '[0-9]{4}' THEN 'YYYY'
            ELSE 'missing'
          end
ORDER  BY date_status; 

 * postgresql+psycopg2://@/postgres


,date_status,counts
0,missing,472
1,YYYY-MM-DD,528


## Start date of concomitant medication (cm.cmstdtc)

In [12]:
%%sql 

SELECT CASE
         WHEN cmstdtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
         WHEN cmstdtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
         WHEN cmstdtc ~* '[0-9]{4}' THEN 'YYYY'
         ELSE 'missing'
       end      AS date_status,
       Count(*) AS counts
FROM   src.cm
GROUP  BY CASE
            WHEN cmstdtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
            WHEN cmstdtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
            WHEN cmstdtc ~* '[0-9]{4}' THEN 'YYYY'
            ELSE 'missing'
          end
ORDER  BY date_status; 

 * postgresql+psycopg2://@/postgres


,date_status,counts
0,missing,21
1,YYYY,3731
2,YYYY-MM,1813
3,YYYY-MM-DD,2035


## End date of concomitant medication (cm.cmendts)

In [13]:
%%sql

SELECT CASE
         WHEN cmendtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
         WHEN cmendtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
         WHEN cmendtc ~* '[0-9]{4}' THEN 'YYYY'
         ELSE 'missing'
       end      AS date_status,
       Count(*) AS counts
FROM   src.cm
GROUP  BY CASE
            WHEN cmendtc ~* '[0-9]{4}-[0-9]{2}-[0-9]{2}' THEN 'YYYY-MM-DD'
            WHEN cmendtc ~* '[0-9]{4}-[0-9]{2}' THEN 'YYYY-MM'
            WHEN cmendtc ~* '[0-9]{4}' THEN 'YYYY'
            ELSE 'missing'
          end
ORDER  BY date_status; 

 * postgresql+psycopg2://@/postgres


,date_status,counts
0,missing,6902
1,YYYY-MM,4
2,YYYY-MM-DD,694
